In [1]:
# -*- coding: utf-8 -*-
"""
Estratificação final pré-RNA (sem gráficos)
Objetivo: Consolidar todas as análises de erro GNSS antes do treinamento da rede neural.
Inclui:
- Erro latitude, longitude e radial
- Métricas estatísticas completas (MAE, RMSE, IQR, etc.)
- Comparativo por período (manhã x tarde)
- Correlação e distribuição por faixa
Base: Dataset_Vetor.json
Autor: Bruno Roglin (TCC UTFPR)
"""

import json
import math
import numpy as np
import pandas as pd
from pathlib import Path

# =========================
# CONFIGURAÇÃO
# =========================
JSON_PATH = "Dataset_Vetor.json"
OUTPUT_DIR = Path("./estratificacao_final_pre_rna")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

lat_col = "diferencalatitudeMetros"
lon_col = "diferencalongitudeMetros"

# =========================
# LEITURA E LIMPEZA
# =========================
print("Carregando dataset...")
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(f"✅ Registros totais: {len(df):,}")

# Verificação de colunas
required = [lat_col, lon_col, "timestamp"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Faltando colunas obrigatórias: {missing}")

# Conversões numéricas
for c in required:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=required)

# Criação de coluna do erro radial e período (manhã < 0.5 / tarde >= 0.5)
df["erro_radial_m"] = np.sqrt(df[lat_col]**2 + df[lon_col]**2)
df["periodo"] = np.where(df["timestamp"] < 0.5, "manhã", "tarde")

print(f"✅ Registros válidos: {len(df):,} | manhã: {len(df[df.periodo=='manhã']):,} | tarde: {len(df[df.periodo=='tarde']):,}")

# =========================
# FUNÇÕES AUXILIARES
# =========================
def resumo_metricas(series: pd.Series) -> dict:
    """Resumo estatístico completo de uma variável numérica."""
    y = pd.to_numeric(series, errors="coerce").dropna()
    n = len(y)
    if n == 0:
        return {}
    q = y.quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
    mean = y.mean()
    std = y.std(ddof=1)
    var = y.var(ddof=1)
    mad = (y - mean).abs().mean()
    mae = y.abs().mean()
    mse = (y**2).mean()
    rmse = math.sqrt(mse)
    iqr = q.loc[0.75] - q.loc[0.25]
    return {
        "n": n,
        "mean": mean,
        "median": q.loc[0.5],
        "std": std,
        "var": var,
        "mad": mad,
        "iqr": iqr,
        "min": y.min(),
        "q1": q.loc[0.25],
        "q3": q.loc[0.75],
        "max": y.max(),
        "range": y.max() - y.min(),
        "mae": mae,
        "mse": mse,
        "rmse": rmse,
    }

def faixa_erro(series: pd.Series) -> pd.DataFrame:
    """Distribuição por faixas absolutas de erro."""
    abs_y = series.abs()
    bins = [-np.inf, 0.5, 1, 2, 3, 5, 10, 20, 50, 100, np.inf]
    labels = ["≤0.5", "0.5–1", "1–2", "2–3", "3–5", "5–10", "10–20", "20–50", "50–100", ">100"]
    dist = pd.cut(abs_y, bins=bins, labels=labels, include_lowest=True)
    freq = dist.value_counts().reindex(labels).fillna(0).astype(int)
    perc = (freq / len(series) * 100).round(3)
    return pd.DataFrame({"FaixaErro(m)": labels, "Quantidade": freq.values, "Percentual(%)": perc.values})

def correlacao_lat_lon(lat, lon):
    """Correlação linear e covariância entre erros."""
    lat = pd.to_numeric(lat, errors="coerce")
    lon = pd.to_numeric(lon, errors="coerce")
    mask = lat.notna() & lon.notna()
    lat, lon = lat[mask], lon[mask]
    pearson = lat.corr(lon, method="pearson")
    spearman = lat.corr(lon, method="spearman")
    return {
        "pearson_r": pearson,
        "spearman_rho": spearman,
        "var_lat": lat.var(ddof=1),
        "var_lon": lon.var(ddof=1),
        "cov_lat_lon": np.cov(lat, lon, ddof=1)[0, 1]
    }

# =========================
# ANÁLISES GERAIS
# =========================
print("\n📊 Calculando métricas globais...")
geral = {
    "Latitude": resumo_metricas(df[lat_col]),
    "Longitude": resumo_metricas(df[lon_col]),
    "ErroRadial": resumo_metricas(df["erro_radial_m"])
}
geral_df = pd.DataFrame(geral)
geral_df.to_csv(OUTPUT_DIR / "metricas_globais.csv")
print(geral_df)

# Distribuição por faixa (global)
lat_faixas = faixa_erro(df[lat_col])
lon_faixas = faixa_erro(df[lon_col])
rad_faixas = faixa_erro(df["erro_radial_m"])

lat_faixas.to_csv(OUTPUT_DIR / "faixas_latitude.csv", index=False)
lon_faixas.to_csv(OUTPUT_DIR / "faixas_longitude.csv", index=False)
rad_faixas.to_csv(OUTPUT_DIR / "faixas_radial.csv", index=False)

# Correlação global
corr = correlacao_lat_lon(df[lat_col], df[lon_col])
pd.DataFrame([corr]).to_csv(OUTPUT_DIR / "correlacao_lat_lon.csv", index=False)
print("\n📈 Correlação LAT×LON:", corr)

# =========================
# ANÁLISES POR PERÍODO (MANHÃ x TARDE)
# =========================
print("\n⏰ Estratificação por período (manhã vs tarde)...")

res_periodo = []
for periodo in ["manhã", "tarde"]:
    sub = df[df["periodo"] == periodo]
    lat_stats = resumo_metricas(sub[lat_col])
    lon_stats = resumo_metricas(sub[lon_col])
    rad_stats = resumo_metricas(sub["erro_radial_m"])
    res_periodo.append({
        "Período": periodo,
        **{f"lat_{k}": v for k, v in lat_stats.items()},
        **{f"lon_{k}": v for k, v in lon_stats.items()},
        **{f"rad_{k}": v for k, v in rad_stats.items()},
    })

periodos_df = pd.DataFrame(res_periodo)
periodos_df.to_csv(OUTPUT_DIR / "metricas_por_periodo.csv", index=False)
print(periodos_df.T)

# Comparativo percentual entre manhã e tarde
manha = periodos_df.loc[periodos_df["Período"] == "manhã"].iloc[0]
tarde = periodos_df.loc[periodos_df["Período"] == "tarde"].iloc[0]

comparativo = []
for key in manha.keys():
    if key == "Período": 
        continue
    m_val, t_val = manha[key], tarde[key]
    if isinstance(m_val, (int, float)) and m_val != 0:
        dif = t_val - m_val
        perc = (dif / abs(m_val)) * 100
        comparativo.append({"Métrica": key, "Diferença": round(dif, 5), "Diferença(%)": round(perc, 3)})

comparativo_df = pd.DataFrame(comparativo)
comparativo_df.to_csv(OUTPUT_DIR / "comparativo_manha_tarde.csv", index=False)

print("\n📘 Comparativo percentual entre períodos salvo em 'comparativo_manha_tarde.csv'")

# =========================
# RESUMO FINAL CONSOLIDADO
# =========================
summary = {
    "Total_amostras": len(df),
    "Amostras_manha": len(df[df.periodo == "manhã"]),
    "Amostras_tarde": len(df[df.periodo == "tarde"]),
    "MAE_lat_global(m)": geral["Latitude"]["mae"],
    "RMSE_lat_global(m)": geral["Latitude"]["rmse"],
    "MAE_lon_global(m)": geral["Longitude"]["mae"],
    "RMSE_lon_global(m)": geral["Longitude"]["rmse"],
    "MAE_radial_global(m)": geral["ErroRadial"]["mae"],
    "RMSE_radial_global(m)": geral["ErroRadial"]["rmse"],
    "Correlação_Pearson_LAT×LON": corr["pearson_r"],
    "Correlação_Spearman_LAT×LON": corr["spearman_rho"],
}
pd.DataFrame([summary]).to_csv(OUTPUT_DIR / "resumo_final.csv", index=False)

print("\n✅ Estratificação final concluída.")
print("💾 Resultados salvos em:", OUTPUT_DIR.resolve())


Carregando dataset...
✅ Registros totais: 419,878
✅ Registros válidos: 419,878 | manhã: 214,258 | tarde: 205,620

📊 Calculando métricas globais...
             Latitude      Longitude     ErroRadial
n       419878.000000  419878.000000  419878.000000
mean         2.542600       0.102123       4.615833
median       2.110000      -0.100000       3.206649
std          5.414513       6.717824       7.721101
var         29.316953      45.129153      59.615402
mad          2.907784       2.270169       2.975400
iqr          4.190000       2.440000       3.600446
min        -87.880000    -124.370000       0.250000
q1           0.510000      -1.210000       1.971852
q3           4.700000       1.230000       5.572298
max         78.200000     134.160000     152.591597
range      166.080000     258.530000     152.341597
mae          3.550320       2.261080       4.615833
mse         35.781699      45.139475      80.921174
rmse         5.981781       6.718592       8.995620

📈 Correlação LAT×LON